In [1]:
# !pip install transformers
# !pip install accelerate


import os
# os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, AutoTokenizer
from tqdm import tqdm
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
data = pd.read_csv('./data/train.csv')

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./alpha_4_29_epoch/", eos_token='</s>')
model = AutoModelForCausalLM.from_pretrained("./alpha_4_29_epoch/")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
formatted_data = []
for _, row in tqdm(data.iterrows()):
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            # 질문과 답변 쌍을 </s> token으로 연결
            input_text = row[q_col] + tokenizer.eos_token + row[a_col] + tokenizer.eos_token
            input_ids = tokenizer.encode(input_text, return_tensors='pt')
            formatted_data.append(input_ids)

644it [00:02, 249.86it/s]


In [6]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 4096)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-27): 28 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)

In [7]:
# for name, module in model.named_children():
#     if name != "embed_out":
#         for param in module.parameters():
#             param.requires_grad = False

In [8]:
for name, module in model.named_children():
    if name == "gpt_neox":
        for name, sub_module in module.named_children():
            # print(name)
            if name == "embed_in":
                for param in sub_module.parameters():
                    param.requires_grad = False
            elif name == "emb_dropout":
                for param in sub_module.parameters():
                    param.requires_grad = False
            elif name == "layers":
                # print(name)
                 for name, layer_module in sub_module.named_children():
                     if int(name) < 22:
                        for param in layer_module.parameters():
                            param.requires_grad = False
            

In [9]:
# model

In [10]:
CFG = {
    'LR' : 7e-6, # Learning Rate
    'EPOCHS' : 30, # 학습 Epoch
}

    
model.to(device) # 모델을 GPU단으로 이동

# 모델 학습 설정
optimizer = AdamW(model.parameters(), lr=CFG['LR'])
model.train()

# 모델 학습
for epoch in range(CFG['EPOCHS']):
    total_loss = 0
    progress_bar = tqdm(enumerate(formatted_data), total=len(formatted_data))
    for batch_idx, batch in progress_bar:
        # 데이터를 GPU단으로 이동
        batch = batch.to(device)
        print(batch)
        print(batch.size())
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # 진행률 표시줄에 평균 손실 업데이트
        progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_idx+1):.4f}")

    # 에폭의 평균 손실을 출력
    print(f"Epoch {epoch+1}/{CFG['EPOCHS']}, Average Loss: {total_loss / len(formatted_data)}")
    model.save_pretrained(f"./alpha_test_{epoch}_epoch")
    tokenizer.save_pretrained(f"./alpha_test_{epoch}_epoch")
# model.save_pretrained("./kakao_10epoch")
# tokenizer.save_pretrained("./kakao_10epoch")

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|                                                                                                                                                                                  | 0/6440 [00:00<?, ?it/s]

tensor([[  378,   473,  4938,   293,  1162,   489,    34, 30003,   378,   473,
          4938,   822, 22311,   363,   540,   272, 10740,  2594,   301,  6709,
         13473,  2592,   274,   458,   272, 10740,   276,  6734,   424,   272,
         10740,  6701,  4938,  1441,    17, 30003]], device='cuda:0')
torch.Size([1, 36])


Epoch 1 - Avg Loss: 0.1265:   0%|                                                                                                                                            | 1/6440 [00:02<3:38:38,  2.04s/it]

tensor([[  378,   473,  4938,   293,  1162,   489,    34, 30003,   378,   473,
          4938,   822,  2592,   285, 22311,   363,  1358,   284,   272, 10740,
          2594,   301,  6709, 13473,  2592,   276,  2378,  4296,    15, 10740,
           276,  6734,   424,   272,  5812,  1441,    17,  3331,  5727,  1019,
          5224, 22311,   285, 10740,   339,  2934,  1461,   301,  3783,   284,
           316,   746,  1203, 10023,    17, 30003]], device='cuda:0')
torch.Size([1, 56])


Epoch 1 - Avg Loss: 0.0971:   0%|                                                                                                                                            | 2/6440 [00:02<1:45:09,  1.02it/s]

...

tensor([[ 4171,  2062,   285,  4591, 14064,   796,   276,  5440,   284,   316,
           992,  4624,  4203,   296,  1662,   423,   272,   904,   909,  6498,
            34, 30003,   346,  2770,   276,  2392,   288,  1607,  2062,   285,
          4591, 14064,   796,  5440,  8717,   296,  4591, 14064,   796,  4729,
          3726,  3474,   274,  3322,  1466,  2288,   285,   955,   524,   533,
          3557,    11,   828,  1192,   377,    15,   469,  1192,   377,    15,
         21633,   377,    12,   301,  2156, 10562,  1912,  2136,    17,  1523,
          1466,  2288,   293,  1399,   482,   586,  1835,   376,  2288,  1751,
          1563,   284,   453,    15,  1415,   678,  2288,   617, 24753,   301,
         10010,   284,   379,  4468,    17,  4764,   469,  1192,   377,    15,
         26001,   377,    15, 21633,   377,  1113,   339, 24753, 10010,  2275,
           293,  1399,   284,   379,  4468,    17, 30003]], device='cuda:0')
torch.Size([1, 117])


Epoch 1 - Avg Loss: 0.0483:   6%|█████████                                                                                                                                   | 417/6440 [01:57<29:22,  3.42it/s]

tensor([[ 4171,  2062,   285,  4591, 14064,   796,   276,  5440,   284,   316,
           992,  4624,  4203,   296,  1662,   423,   272,   904,   909,  6498,
            34, 30003,   346,  2770,   276,  2392,   288,  1607,  2062,   285,
          4591, 14064,   796,  5440,  8717,   296,  1754, 10562,  1466,  2288,
           393,   586,   524,   533,  3557,    11,   828,  1192,   377,    15,
           469,  1192,   377,    15, 21633,   377,    12,   301,  2156,   339,
         10010,  2136,    17,  1273,  1466,  2288,   293,  6118,   586,  1835,
           376,  2288,  1751,  1563, 13473,  1415,   678,  2288,   617,  5440,
          8717,   276,  1381,  2136,    17,  4764,   469,  1192,   377,    15,
         26001,   377,    15, 21633,   377,  7940,   286,  1563,   284,   379,
          4468,    17,  3322,  1186,   288,  2275,   285,  3110,   441, 14280,
           433,   274,  4845,  1721,   276,  2146, 13473,  4591, 14064,   796,
          5440,   270,  8295,   379,  4468,    17, 3

Epoch 1 - Avg Loss: 0.0483:   6%|█████████                                                                                                                                   | 417/6440 [01:58<28:26,  3.53it/s]


KeyboardInterrupt: 